# Project UPDATE for ViT Hybrid (ConvStems) Setup:

### Architecture
Run pre-training with contrastive learning (MoCo framework)
* Use backbone - `ViT Hybrid`
* **Perform SSL Pre-training** of backbone using contrastive learning (MoCo) -> augment medical imaging dataset (Chest XRs) (Pre-text task); create positive and negative pairs
  * Produce: `moco_vit_hybrid_encoder.pth`
* **Transfer Learning:** Fine-tune pre-trained ResNet for Pneumonia Chest XR classification
  * Produce: `finetuned_vit_hybrid_medical.pth`

### Pre-training Dataset: CheXpert
* Subset: Pneumonia classification only; smaller dataset (to accomodate class imbalance)

### Fine-tuning Dataset: NIH Pneumonia Dataset

###

In [1]:
# import libraries
import os
import sys
import argparse
from tqdm import tqdm

import numpy as np
import seaborn as sns
import pandas as pd
import math
from copy import deepcopy

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display

import pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Collab Needs

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define filepaths to required input (scripts, data) and outputs

In [3]:
# ----------------------------------------------------
# Inputs root
# ----------------------------------------------------
FP_ROOT="/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/"

# ----------------------------------------------------
# Dataset info
# ----------------------------------------------------
# Chexpert
DATASET_PATH_SPLIT = FP_ROOT + "Data/CheXpert_reduced_dataset_split_v3.zip"

# ChexPert
TRAIN_LABELS_CSV = FP_ROOT + "Data/1_final_project_updated_names_train_moco.csv"
TEST_LABELS_CSV = FP_ROOT + "Data/1_final_project_updated_names_test_moco.csv"

LINEAR_TRAIN_LABELS_CSV = FP_ROOT + "Data/2_final_project_updated_names_train_linear.csv"
LINEAR_TEST_LABELS_CSV = FP_ROOT + "Data/2_final_project_updated_names_test_linear.csv"

# ----------------------------------------------------
# Model SRC
# ----------------------------------------------------
SRC_ROOT = FP_ROOT + "src/"
TRAIN_SCRIPT = f"{SRC_ROOT}/train_moco.py"
MOCO_FOLDER = f"{SRC_ROOT}/moco"

# ----------------------------------------------------
# Outputs
# ----------------------------------------------------
ROOT_ARTIFACT_SAVE = FP_ROOT + "artifacts/"

In [4]:
# Add the project /src to the system path
if SRC_ROOT not in sys.path:
    sys.path.append(SRC_ROOT)
    print(f"Added {SRC_ROOT} to sys.path")

Added /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src/ to sys.path


In [5]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpl1gxaaah', '/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src/']


## Unzip data

ChexPert

In [6]:
# Unzip the dataset (image) files to /tmp
# DATA_DEST_UNZIPPED = "/tmp/CheXpert_dataset/"
# os.makedirs(DATA_DEST_UNZIPPED, exist_ok=True)
# !unzip "{DATASET_PATH_SPLIT}" -d {DATA_DEST_UNZIPPED} # need to use "" to accomdate space

In [7]:
# Update for the unzipped sub-name
DATA_DEST_UNZIPPED = "/tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_v3/"

## 1) Run Pre-training - Contrastive Learning

In [8]:
# Run from the src directory
%cd "/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src"

/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src


In [9]:
# Hyperparams
batch_size = 32

In [10]:
torch.cuda.empty_cache()

In [ ]:
! python moco/train_moco_vit_hybrid.py \
    --train_csv_path "$TRAIN_LABELS_CSV" \
    --test_csv_path "$TEST_LABELS_CSV" \
    --root_dir "$DATA_DEST_UNZIPPED" \
    --artifact_root "$ROOT_ARTIFACT_SAVE" \
    --batch_size "$batch_size" \
    --n_epochs 25 \
    --test_num_classes 2 \
    --linear_n_epochs 10 \
    --linear_train_csv_path "$LINEAR_TRAIN_LABELS_CSV" \
    --linear_test_csv_path "$LINEAR_TEST_LABELS_CSV" \
    --model_type "VIT_Hybrid" \
    --out_model_name "vit_hybrid_moco_encoder.pth" \
    --embedding_dim 192

Created log file:  /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts/moco_training_log_20251201_062301.txt
[2025-12-01 06:23:01] Creating MoCo medical image Train DataLoader... : from /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/1_final_project_updated_names_train_moco.csv
Loading dataset from: /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/1_final_project_updated_names_train_moco.csv
Training Dataset size: 49500 images
[2025-12-01 06:23:01] Using ViT Hybrid backbone for MoCo
[2025-12-01 06:23:01] Note: No option to use pretrained weights; ignoring if set
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
[2025-12-01 06:23:02] Starting MoCo ViT Hybrid training for 50 epochs
Epoch 0/49: 100% 1546/1546 [38:21<00:00,  1.49s/it, loss=7.7